** Simplified version of previous notebook. Only COD 900 considered as the possible maximum energy stored by the system. 

In [22]:
import numpy as np
import pandas as pd

#### Size system components e.g. capacitor given the amount of energy needed to be stored. 

#### Find the maximum and minimum net energy available given different COD and storage capacitors. 

In [151]:
# Energy harvested from single batch feed MFC, Joules (J) for different COD values

Eave = np.array([21.137,  # COD = 900
                 18.017,  # COD = 500
                 8.315,   # COD = 300
                 2.415    # COD = 70
                ])

Eave *= 0.48 # apply energy harvesting eff.
Eave[0]

21.137

Maximum and minimum rated supply voltages (e.g. for regulator to supply 1.8V) for power management components.

Two system configurations (different combinations of components considerd) 

In [152]:
# max min allowable supply voltage for comparator and voltage regulator
# comp:LTC1540, reg:LT3009

Vmax = 11
Vmin = 2.3

In [153]:
# Create a data frame for each system config 
E_cod900 = Eave[0]
E_cod900

21.137

Capacitor energy $E = \frac{1}{2} CV^2$

Capacitance needed to store `E_cod900` (average energy harvetsed at each COD value) at Vmax found by:

$C = \frac{E_{max}}{\frac{1}{2} V_{max}^2}$

In [154]:
# capacitor needed to store Emax for each COD value at Vmax for each system config

C = E_cod900 / (0.5 * Vmax**2)
C = round(C, 3)
print(f'capacitor = {C} F')

capacitor = 0.349 F


For a capacitor of this size, the remaining energy $E_{min}$ stored in the capacitor when drained to the minimum allowable supply voltage for the system $V_{min}$ found by: 

$E_{min} = \frac{1}{2} CV_{min}^2$

In [155]:
# Average energy harvested at lowest COD value tested (COD = 70, 900)

Emin = 0.5 * C * Vmin**2
Emin

0.9231049999999998

For each COD value, we can find the available energy `Ea`:

$E_{a} = E_{ave} - E_{min}$


In [156]:
Ea = Eave - Emin
Ea

array([20.213895, 17.093895,  7.391895,  1.491895])

In [157]:
#array([9.7014 , 8.2038 , 3.54684, 0.71484])

#### $E_a$ determines how many samples can be taken by the data logger in the next week/cycle if the energy consumed by the other processes is known. 

$E_{a} = nE_{sa} + E_{e} + E_{sl} + E_{r} + E_{c}$

where $n=$ number of samples 

#### Use the energy consumption of each process done by the microcontroller and power management system to calculate the allowable number of samples corresponding to each COD value 

Estimated voltage of the storagae capacitor for each COD value

In [158]:
def cap_v(E, C):
    return np.sqrt(2 * E / C)

V_cod70 = cap_v(Eave[3], C)
V_cod300 = cap_v(Eave[2], C)
V_cod500 = cap_v(Eave[1], C)
V_cod900 = cap_v(Eave[0], C)

print(V_cod70, V_cod300, V_cod500, V_cod900)


# Average voltage of the capacitor 

3.72015343060981 6.902929073909635 10.161165467977666 11.005859340736652


In [159]:
from scipy.integrate import quad

def V_ave(V0, R=3333333):
    Vc = lambda t : V0 * np.exp(-t/(R*C))
    #t_ = - R * C * np.log( Vmin / V0 )
    week = 604800 # # s
    week = 432000 
    Vc_tot = V0 * quad(lambda t : np.exp(-t/(R * C)), 0, week)[0]
    print(Vc_tot)
    Vave = Vc_tot / week
    return(Vave)

print(V_ave(V_cod70), V_ave(V_cod300), V_ave(V_cod500), V_ave(V_cod900))

1342456.3446392461
2490994.284164692
3666763.0262711514
3971579.6608305806
3.1075378348130696 5.766190472603453 8.487877375627665 9.193471437107826


In [160]:
processes = ['sample', 'estimate', 'sleep', 'regulator', 'comparator']

# execution time (s)
week = 7*24*60*60
week = 432000 
exec_time = [0.1, 0.3, week, week, week]

# current 
# microcontroller operations measured empirically
# comparator, regulator max current from data sheet
current = np.array([1.18e-3, 2.85e-3, 2e-8, 3e-6, 2.8e-7]) 


# voltage 
Vmc = 1.8 # operting voltage of microcontroller 
voltage = np.array([[Vmc, Vmc, Vmc, V_ave(V_cod900), V_ave(V_cod900)],
                    [Vmc, Vmc, Vmc, V_ave(V_cod500), V_ave(V_cod500)],
                    [Vmc, Vmc, Vmc, V_ave(V_cod300), V_ave(V_cod300)],
                    [Vmc, Vmc, Vmc, V_ave(V_cod70),  V_ave(V_cod70)]])

# power
print()
power = voltage * current
print(np.round(power,8))
print()

# energy 
energy = power * exec_time 
print(np.round(energy,4))

3971579.6608305806
3971579.6608305806
3666763.0262711514
3666763.0262711514
2490994.284164692
2490994.284164692
1342456.3446392461
1342456.3446392461

[[2.124e-03 5.130e-03 4.000e-08 2.758e-05 2.570e-06]
 [2.124e-03 5.130e-03 4.000e-08 2.546e-05 2.380e-06]
 [2.124e-03 5.130e-03 4.000e-08 1.730e-05 1.610e-06]
 [2.124e-03 5.130e-03 4.000e-08 9.320e-06 8.700e-07]]

[[2.00000e-04 1.50000e-03 1.56000e-02 1.19147e+01 1.11200e+00]
 [2.00000e-04 1.50000e-03 1.56000e-02 1.10003e+01 1.02670e+00]
 [2.00000e-04 1.50000e-03 1.56000e-02 7.47300e+00 6.97500e-01]
 [2.00000e-04 1.50000e-03 1.56000e-02 4.02740e+00 3.75900e-01]]


average voltage in capacitor for each COD 

https://tutorial.math.lamar.edu/Classes/CalcI/AvgFcnValue.aspx

$E_{out} = nE_{sa} + E_{e} + E_{sl} + E_{r} + E_{c}$

where $n=$ number of samples 

Minimum energy for one cycle ($n=1$) 

In [161]:
En1 = np.sum(energy, axis=1)
En1

array([13.04408469, 12.04428613,  8.18776465,  4.42056021])

Plot initial energy against number of samples and do table of n samlpes and samplin interval for each COD value 